In [1]:
"""Colocar searchtags"""

'Colocar searchtags'

In [2]:
"""Para setar localização:



from django.contrib.gis.geos import Point, GeometryCollection

geometry=GeometryCollection([Point()])


"""

'Para setar localização:\n\n\n\nfrom django.contrib.gis.geos import Point, GeometryCollection\n\ngeometry=GeometryCollection([Point()])\n\n\n'

In [3]:
"""
record=OcurrenceRecord()
record.save()

service_order=ServiceOrder(company=company, description=texto_excel,kind="LAND",process_type="1",)
service_order.save()

record.service_orders.add(service_order)
service_order.responsibles.add(User.objects.get(...))

service_order_action=ServiceOrderAction(name=fazer loop em opções da company.metadata, criando os itens abaixo da escolha do excel
                    (incluindo), em ordem, estimated_end_date=date.today()+timedelta(week=valor no loop),
                    created_by=engie, service_order=service_order)
service_order_action.save()

procedure=Procedure(action=service_order_action,responsible=User.objects.get(...),to_do=texto_excel,
                    created_by=engie)

"""

'\nrecord=OcurrenceRecord()\nrecord.save()\n\nservice_order=ServiceOrder(company=company, description=texto_excel,kind="LAND",process_type="1",)\nservice_order.save()\n\nrecord.service_orders.add(service_order)\nservice_order.responsibles.add(User.objects.get(...))\n\nservice_order_action=ServiceOrderAction(name=fazer loop em opções da company.metadata, criando os itens abaixo da escolha do excel\n                    (incluindo), em ordem, estimated_end_date=date.today()+timedelta(week=valor no loop),\n                    created_by=engie, service_order=service_order)\nservice_order_action.save()\n\nprocedure=Procedure(action=service_order_action,responsible=User.objects.get(...),to_do=texto_excel,\n                    created_by=engie)\n\n'

In [4]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [5]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: 4a5e0119-fb85-40fd-87ab-f201a2aa0fde: GSB - UHE-MACHADINHO>, <Company: 7fe7b473-6d9c-49d6-a9ca-74f3424d3211: GSB - UHE-SALTO OSORIO>, <Company: b75339ff-e91f-4e75-a082-5d7e23a4d049: GSB - UHE-SALTO SANTIAGO>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: GSO - UHE-JAGUARA>, <Company: d25d0eb2-61a8-4646-8e0e-31c8e68cf216: TESTE>, <Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: GSO - UHE-CANA BRAVA>, <Company: 62ad80d9-b785-4037-933a-e23238728220: GSO - UHE-MIRANDA>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: GSO - UHE-SALTO SANTIAGO>, <Company: cff59760-69dd-4d07-bba5-61aa8b6faeea: GES - UHE-CANA BRAVA>, <Company: 90041917-e41e-46f2-b979-7d3ad53efad2: Engie Umburanas>, <Company: 7647aa61-e7eb-42f4-b2c1-5567d083df4f: Energia Compartilhado GSO>, <Company: 4b4bfdad-

In [6]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta, date, datetime
import random
from copy import deepcopy
import math
import locale
from tqdm.notebook import tqdm
import pytz
from django.contrib.gis.geos import Point, GeometryCollection
import subprocess
import sys
from django.contrib.admin.utils import flatten
from helpers.strings import get_obj_from_path, keys_to_snake_case, to_snake_case, UF_CODE
from uuid import uuid4
from django.contrib.gis.geos import Point

In [7]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [8]:
def get_inner_value(field, inner_field, occ, value):
    values = occ.form_fields['fields']
#     value=value.capitalize().strip()
    item_translation = flatten([item['inner_fields'] for item in values if item['api_name']==field])
#     print(item_translation)
    mid_translation = flatten([item['select_options']['options'] for item in item_translation if item['display_name'] == inner_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def property_intersections(instance):
    geom = instance.geometry

    if geom and "properties_shape" in instance.company.metadata:
        try:
            shape = ShapeFile.objects.get(
                uuid=instance.company.metadata["properties_shape"]
            )
        except Exception:
            pass
        else:
            intersects = []
            for index, geometry in enumerate(shape.geometry):
                try:
                    if geometry.intersects(geom):
                        intersects.append((index, geometry))
                except Exception:
                    pass

            intersects = [
                {
                    "geometry": json.loads(geometry[1].json),
                    "attributes": {
                        "uuid": "{}-{}".format(
                            str(shape.uuid),
                            shape.properties[geometry[0]]["OBJECTID"],
                        ),
                        **shape.properties[geometry[0]],
                    },
                }
                for geometry in intersects
            ]

            field = "property_intersections"
            instance.form_data[field] = intersects

In [28]:
filename='UHPF'

In [29]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
# sheetname
ws = wb[sheetname]

In [30]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

In [31]:
values[1]

{'N. da Invasão': None,
 'Tipo Relatório': 'RO',
 'Relatório Principal': 201000295,
 'Região': '00',
 'Fuso': '22S',
 'Coordenada X': 330706,
 'Coordenada Y': 6953497,
 'Município': 'Entre Rios do Sul - RS',
 'Status': 'Atendido',
 'Equipe': 'LAGO AZUL',
 'Data do registro': datetime.datetime(2009, 4, 3, 0, 0),
 'Classe': 'Uso e Ocupação Irregular',
 'Identificação do local_1': 'Área de preservação permanente',
 'Identificação do local_2': None,
 'Identificação do local_3': None,
 'Identificação do local_4': None,
 'Identificação do local_5': None,
 'Identificação do local_6': None,
 'Identificação do local_7': None,
 'Identificação do local_8': None,
 'Constatação_1': 'Ocupação/utilização não autorizada',
 'Constatação_2': None,
 'Constatação_3': None,
 'Constatação_4': None,
 'Constatação_5': None,
 'Constatação_6': None,
 'Contatação_7': None,
 'Constatação_8': None,
 'Constatação (detalhe)_1': None,
 'Irregularidades_1': 'lavoura ',
 'Irregularidades_2': 'edificação ',
 'Irregulari

In [32]:
company = Company.objects.get(name='GES - UHE-PASSO FUNDO')

engie = User.objects.get(username="engie")

In [35]:
occ=OccurrenceType.objects.get(uuid='faa62cbd-c2ec-49fc-bc4a-9b1ab9edcca4')# Ocorrências Patrimoniais
occ_RE=OccurrenceType.objects.get(uuid='df7b367d-893b-40cc-af39-2019a2e764b9')# Resumo Executivo

user=User.objects.get(username='engie') #user criador dos registros

firm=Firm.objects.get(company=company,name='LAGO AZUL')
tag1=SearchTag.objects.get(company = company, name = "Ocorrência")
tag3=SearchTag.objects.get(company = company, name = "Patrimonial")
tag4=SearchTag.objects.get(company = company, name = "Uso e ocupação irregular")
actions=get_obj_from_path(company.metadata, "land_action_names")
status = ServiceOrderActionStatus.objects.get(name='Atendido', companies=company)


deadline = '22/04/2024 00:00'
path='importacao_2024/uhso/'

array_name='irregularities'
occ,occ_RE

(<OccurrenceType: Ocorrências patrimoniais - ['GES - UHE-SALTO SANTIAGO', 'GES - UHE-ITA', 'GES - GEH', 'GES - UHE-ESTREITO', 'GES - UHE-MIRANDA', 'GES - UHE-SALTO OSORIO', 'GES - UHE-CANA BRAVA', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-PASSO FUNDO', 'GES - UHE-MACHADINHO', 'GES - UHE-PONTE DE PEDRA', 'GES - PCH-JOSÉ GELÁZIO DA ROCHA', 'GES - PCH-RONDONÓPOLIS', 'GES - UHE-JAGUARA']>,
 <OccurrenceType: Resumo Executivo - ['GES - UHE-SALTO SANTIAGO', 'GES - UHE-ITA', 'GES - GEH', 'GES - UHE-ESTREITO', 'GES - UHE-JAGUARA', 'GES - UHE-MIRANDA', 'GES - UHE-SALTO OSORIO', 'GES - UHE-CANA BRAVA', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-PASSO FUNDO', 'GES - UHE-MACHADINHO', 'GES - UHE-PONTE DE PEDRA', 'GES - PCH-JOSÉ GELÁZIO DA ROCHA', 'GES - PCH-RONDONÓPOLIS']>)

In [36]:
tag1.name,tag3.name,tag4.name

('Ocorrência', 'Patrimonial', 'Uso e ocupação irregular')

In [37]:
tag10=SearchTag.objects.get(company = company, name = "Relatório")
tag30=SearchTag.objects.get(company = company, name = "Resumo executivo")
tag40=SearchTag.objects.get(company = company, name = "Patrimonial")
tag10.name,tag30.name,tag40.name

('Relatório', 'Resumo executivo', 'Patrimonial')

In [38]:
def get_action_list(actions, value):
    value_index = 0
    for index, a in enumerate(actions):
        if a['name'] == value:
            value_index=index
            break

    new_actions=actions[value_index:]
    
    return new_actions

In [39]:
def transform_coordinates(x, y, srid=31982):

    pnt = Point(x, y, srid=srid)
    pnt.transform(4326)

    lng = pnt.coords[0]
    lat = pnt.coords[1]
    return Point(lng, lat)

In [40]:
def get_uf(uf_code):
    if type(uf_code) != 'str':
        uf_code=str(uf_code)
    
    uf=[{'id': '12', 'name': 'AC'}, {'id': '27', 'name': 'AL' }, {'id': '13', 'name': 'AM'}, {'id': '16', 'name': 'AP'}, 
        {'id': '29', 'name': 'BA'}, {'id': '23', 'name': 'CE' }, {'id': '53', 'name': 'DF'}, {'id': '32', 'name': 'ES'}, 
        {'id': '52', 'name': 'GO'}, {'id': '21', 'name': 'MA' }, {'id': '31', 'name': 'MG'}, {'id': '50', 'name': 'MS'}, 
        {'id': '51', 'name': 'MT'}, {'id': '15', 'name': 'PA' }, {'id': '25', 'name': 'PB'}, {'id': '26', 'name': 'PE'}, 
        {'id': '22', 'name': 'PI'}, {'id': '41', 'name': 'PR' }, {'id': '33', 'name': 'RJ'}, {'id': '24', 'name': 'RN'},
        {'id': '11', 'name': 'RO'}, {'id': '14', 'name': 'RR' }, {'id': '43', 'name': 'RS'}, {'id': '42', 'name': 'SC'},
        {'id': '28', 'name': 'SE'}, {'id': '35', 'name': 'SP' }, {'id': '17', 'name': 'TO'}]
    uf_translation={item['name']: item['id'] for item in uf}
#     print(uf_translation)    
    
    return uf_translation[uf_code]

In [41]:
def get_involved_parts(a):
    return_dict = list(filter(None, [{
        'involved_parts': '1',
        'name': a['Nome do infrator'],
        'rg': True if a.get('RG') is not None else False,
        'description_rg': str(a.get('RG')).strip() if a.get('RG') is not None else None,
        'cpf': True if a.get('CPF / CNPJ') is not None else False,
        'description_cpf': str(a.get('CPF / CNPJ')).strip() if a.get('CPF / CNPJ') is not None else None,
        'phone': True if a.get('Telefone_1') is not None else False,
        'description_phone': str(a.get('Telefone_1')).strip() if a.get('Telefone_1') is not None else None,
        'email': True if a.get('E-mail') is not None else False,
        'description_email': a.get('E-mail').strip() if a.get('E-mail') is not None else None,
        'address': a.get('Logradouro').strip() if a.get('Logradouro') is not None else None,
        'address_number': str(a.get('Número')) if a.get('Número') is not None else None,
        'district': a.get('Bairro').strip() if a.get('Bairro') is not None else None,
        'address_additional': a.get('Complemento').strip() if a.get('Complemento') is not None else None,
#         'address_additional': a['Complemento'] if a['Complemento'] is not None else None,
        'uf': int(get_uf(a.get('UF').strip())) if a.get('UF') is not None else None,
        'city': a.get('Cidade').strip() if a.get('Cidade') is not None else None,
        'cep': str(a.get('CEP')).strip() if a.get('CEP') is not None else None
    }]))
    
    return return_dict

In [42]:
for a in tqdm(values):
    if not a.get('Observações_1'):
        continue
    a['obs'] = list(filter(None, [{'observation_description': a['Observações_1']} if a['Observações_1'] else None,
                                 {'observation_description': a['Observações_2']} if a['Observações_2'] else None,
                                 {'observation_description': a['Observações_3']} if a['Observações_3'] else None,
                                 {'observation_description': a['Observações_4']} if a['Observações_4'] else None,
                                 {'observation_description': a['Observações_5']} if a['Observações_5'] else None,]))

  0%|          | 0/427 [00:00<?, ?it/s]

In [43]:
for a in tqdm(values):
    a['irreg'] = list(filter(None,[{'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_1']) if a['Identificação do local_1'] is not None else None,
                   'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_1']) if a['Constatação_1'] is not None else None,
                   'place_identification': a['Constatação (detalhe)_1'].capitalize().strip() if a['Constatação (detalhe)_1'] is not None else None,
                   'unauthorized_construction_irregular': a['Irregularidades_1'].strip().capitalize() if a.get('Irregularidades_1') is not None else None,
                   'other_description': a['Outros - Descrição_1'] if a.get('Outros - Descrição_1') is not None else None,
#                    'display_length': True if a['Comprimento (m)_1'] is not None else False,
#                    'length': a['Comprimento (m)_1'] if a['Comprimento (m)_1'] is not None else None,
#                    'display_width': True if a['Largura (m)_1'] is not None else False,
#                    'width': a['Largura (m)_1'] if a['Largura (m)_1'] is not None else None,
#                    'display_height': True if a['Altura (m)_1'] is not None else False,
#                    'height': a['Comprimento (m)_1'] if a['Altura (m)_1'] is not None else None,
#                    'display_area': True if a['Área (m2)_1'] is not None else False,
#                    'area': a['Área (m2)_1'] if a['Área (m2)_1'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_1'] is not None else False,
#                    'amount': a['Quantidade (un)_1'] if a['Quantidade (un)_1'] is not None else None,
#                    'display_volume': True if a['Volume (m3)_1'] is not None else False,
#                    'volume': a['Volume (m3)_1'] if a['Volume (m3)_1'] is not None else None,
#                    'display_deepen': True if a['Profundidade (m)_1'] is not None else False,
#                    'deepen': a['Profundidade (m)_1'] if a['Profundidade (m)_1'] is not None else None,
#                    'display_perimeter': True if a['Perímetro (m)_1'] is not None else False,
#                    'perimeter': a['Diâmetro (m)_1'] if a['Perímetro (m)_1'] is not None else None,
#                    'display_diameter': True if a['Diâmetro (m)_1'] is not None else False,
#                    'diameter': a['Diâmetro (m)_1'] if a['Diâmetro (m)_1'] is not None else None,
#                    'display_quote': True if a['Cota (m)_1'] is not None else False,
#                    'quote': a['Cota (m)_1'] if a['Cota (m)_1'] is not None else None,
#                    'display_kg': True if a['Peso (kg)_1'] is not None else False,
#                    'weight': a['Peso (kg)_1'] if a['Peso (kg)_1'] is not None else None                  
                  },
                 {
#                 'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_2']) if a['Identificação do local_2'] is not None else None,
#                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_2']) if a['Constatação_2'] is not None else None,
#                    'place_identification': a['Constatação (detalhe)_2'] if a['Constatação (detalhe)_2'] is not None else None,
                   'unauthorized_construction_irregular': a['Irregularidades_2'].strip().capitalize() if a['Irregularidades_2'] is not None else None,
#                    'other_description': a['Outros - Descrição_2'] if a['Outros - Descrição_2'] is not None else None,
#                    'display_length': True if a['Comprimento (m)_2'] is not None else False,
#                    'length': a['Comprimento (m)_2'] if a['Comprimento (m)_2'] is not None else None,
#                    'display_width': True if a['Largura (m)_2'] is not None else False,
#                    'width': a['Largura (m)_2'] if a['Largura (m)_2'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_2'] is not None else False,
#                    'amount': a['Quantidade (un)_2'] if a['Quantidade (un)_2'] is not None else None                 
                 } if a['Constatação_1'] is not None else None,
                 {
#                      'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_3']) if a['Identificação do local_3'] is not None else None,
#                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_3']) if a['Constatação_3'] is not None else None,
#                    'place_identification': a['Constatação (detalhe)_3'] if a['Constatação (detalhe)_3'] is not None else None,
                   'unauthorized_construction_irregular': a['Irregularidades_3'].strip().capitalize() if a['Irregularidades_3'] is not None else None,
#                    'other_description': a['Outros - Descrição_3'] if a['Outros - Descrição_3'] is not None else None,
#                    'display_length': True if a['Comprimento (m)_3'] is not None else False,
#                    'length': a['Comprimento (m)_3'] if a['Comprimento (m)_3'] is not None else None,
#                    'display_width': True if a['Largura (m)_3'] is not None else False,
#                    'width': a['Largura (m)_3'] if a['Largura (m)_3'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_3'] is not None else False,
#                    'amount': a['Quantidade (un)_3'] if a['Quantidade (un)_3'] is not None else None                
                  } if a['Constatação_1'] is not None else None,
                 {
#                      'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_4']) if a['Identificação do local_4'] is not None else None,
#                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_4']) if a['Constatação_4'] is not None else None,
#                    'place_identification': a['Constatação (detalhe)_4'] if a['Constatação (detalhe)_4'] is not None else None,
                   'unauthorized_construction_irregular': a['Irregularidades_4'].strip().capitalize() if a['Irregularidades_4'] is not None else None,
#                    'other_description': a['Outros - Descrição_4'] if a['Outros - Descrição_4'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_4'] is not None else False,
#                    'amount': a['Quantidade (un)_4'] if a['Quantidade (un)_4'] is not None else None  
                  } if a['Constatação_1'] is not None else None,
                 {
#                      'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_5']) if a['Identificação do local_5'] is not None else None,
#                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_5']) if a['Constatação_5'] is not None else None,
#                    'place_identification': a['Constatação (detalhe)_5'] if a['Constatação (detalhe)_5'] is not None else None,
                   'unauthorized_construction_irregular': a['Irregularidades_5'].strip().capitalize() if a['Irregularidades_5'] is not None else None,
#                    'other_description': a['Outros - Descrição_5'] if a['Outros - Descrição_5'] is not None else None,
#                    'display_amount': True if a['Quantidade (un)_5'] is not None else False,
#                    'amount': a['Quantidade (un)_5'] if a['Quantidade (un)_5'] is not None else None             
                  } if a['Constatação_1'] is not None else None,
#                   {
# #                       'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_6']) if a['Identificação do local_6'] is not None else None,
# #                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_6']) if a['Constatação_6'] is not None else None,
#                    'place_identification': a['Constatação (detalhe)_6'] if a['Constatação (detalhe)_6'] is not None else None,
# #                    'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_6']) if a['Irregularidades_6'] is not None else None,
# #                    'other_description': a['Outros - Descrição_6'] if a['Outros - Descrição_6'] is not None else None,
# #                    'display_amount': True if a['Quantidade (un)_6'] is not None else False,
# #                    'amount': a['Quantidade (un)_6'] if a['Quantidade (un)_6'] is not None else None             
#                   } if a['Constatação_1'] is not None else None,
#                   {
# #                       'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_7']) if a['Identificação do local_7'] is not None else None,
# #                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_7']) if a.get('Constatação_7') is not None else None,
#                    'place_identification': a['Constatação (detalhe)_7'] if a['Constatação (detalhe)_7'] is not None else None,
# #                    'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_7']) if a['Irregularidades_7'] is not None else None,
# #                    'other_description': a['Outros - Descrição_7'] if a['Outros - Descrição_7'] is not None else None,
# #                    'display_amount': True if a['Quantidade (un)_7'] is not None else False,
# #                    'amount': a['Quantidade (un)_7'] if a['Quantidade (un)_7'] is not None else None             
#                   } if a['Constatação_1'] is not None else None,
#                   {
# #                       'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_8']) if a['Identificação do local_8'] is not None else None,
# #                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_8']) if a.get('Constatação_8') is not None else None,
#                    'place_identification': a['Constatação (detalhe)_8'] if a['Constatação (detalhe)_8'] is not None else None,
# #                    'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_8']) if a['Irregularidades_8'] is not None else None,
# #                    'other_description': a['Outros - Descrição_8'] if a['Outros - Descrição_8'] is not None else None,
# #                    'display_amount': True if a['Quantidade (un)_8'] is not None else False,
# #                    'amount': a['Quantidade (un)_8'] if a['Quantidade (un)_8'] is not None else None             
#                   } if a['Constatação_1'] is not None else None,
#                   {
# #                       'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_9']) if a['Identificação do local_9'] is not None else None,
# #                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_9']) if a.get('Constatação_9') is not None else None,
#                    'place_identification': a['Constatação (detalhe)_9'] if a['Constatação (detalhe)_9'] is not None else None,
# #                    'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_9']) if a['Irregularidades_9'] is not None else None,
# #                    'other_description': a['Outros - Descrição_9'] if a['Outros - Descrição_9'] is not None else None,
# #                    'display_amount': True if a['Quantidade (un)_7'] is not None else False,
# #                    'amount': a['Quantidade (un)_7'] if a['Quantidade (un)_7'] is not None else None             
#                   } if a['Constatação_1'] is not None else None,
#                   {
# #                       'place_identification': get_inner_value(array_name, 'Identificação do local', occ, a['Identificação do local_10']) if a['Identificação do local_10'] is not None else None,
# #                    'finding': get_inner_value(array_name, 'Constatação', occ, a['Constatação_10']) if a.get('Constatação_10') is not None else None,
#                    'place_identification': a['Constatação (detalhe)_10'] if a['Constatação (detalhe)_10'] is not None else None,
# #                    'unauthorized_construction_irregular': get_inner_value(array_name, 'Irregularidades', occ, a['Irregularidades_10']) if a['Irregularidades_10'] is not None else None,
# #                    'other_description': a['Outros - Descrição_10'] if a['Outros - Descrição_10'] is not None else None,
# #                    'display_amount': True if a['Quantidade (un)_7'] is not None else False,
# #                    'amount': a['Quantidade (un)_7'] if a['Quantidade (un)_7'] is not None else None             
#                   } if a['Constatação_1'] is not None else None
                                  ]
                            ))
    

  0%|          | 0/427 [00:00<?, ?it/s]

In [44]:
# for a in tqdm(values):
#     if a['Data do registro'] is None or a['Data do registro'] == 'None':
#         continue
#     if isinstance(a['Data do registro'],str):
#         a['Data do registro']=a['Data do registro'].strip()
#     if isinstance(a['Data do registro'],datetime):
#         a['Data do registro']=a['Data do registro'].strftime('%Y-%m-%d')
#     if '00:00' in a['Data do registro']:
#         a['Data do registro'] = a['Data do registro'].split(' ')[0]

#     try:
#         a['Data do registro'] = datetime.strptime(a['Data do registro'].strip(), '%Y-%m-%d')
#     except:
#         a['Data do registro'] = datetime.strptime(a['Data do registro'].strip(), '%d/%m/%Y')

In [45]:
User.objects.get(username='fernando.azevedo').get_full_name()

'Fernando Azevedo'

In [47]:
for a in values:
    if a['Município'] == 'Quatro Irmãos':
        a['Município'] = 'Quatro Irmãos - RS'

In [49]:
values[132]

{'N. da Invasão': None,
 'Tipo Relatório': 'RO',
 'Relatório Principal': 201001358,
 'Região': '02',
 'Fuso': '22S',
 'Coordenada X': 326042,
 'Coordenada Y': 6931031,
 'Município': 'Ronda Alta - RS',
 'Status': 'Atendido',
 'Equipe': 'LAGO AZUL',
 'Data do registro': datetime.datetime(2015, 2, 5, 0, 0),
 'Classe': 'Uso e Ocupação Irregular',
 'Identificação do local_1': 'Área de preservação permanente',
 'Identificação do local_2': None,
 'Identificação do local_3': None,
 'Identificação do local_4': None,
 'Identificação do local_5': None,
 'Identificação do local_6': None,
 'Identificação do local_7': None,
 'Identificação do local_8': None,
 'Constatação_1': 'Ocupação/utilização não autorizada',
 'Constatação_2': None,
 'Constatação_3': None,
 'Constatação_4': None,
 'Constatação_5': None,
 'Constatação_6': None,
 'Contatação_7': None,
 'Constatação_8': None,
 'Constatação (detalhe)_1': None,
 'Irregularidades_1': 'movimentação de terra ',
 'Irregularidades_2': None,
 'Irregularida

In [50]:
erros=[]
for i,a in enumerate(tqdm(values[133:])):
    
    engie_user=User.objects.get(username='fernando.azevedo')
    
    try:
        
        """Geometria"""
        properties=[]
        srid = 31982 if '22' in str(a['Fuso']) else 31983
        x = int(str(a['Coordenada X']).strip()) if a['Coordenada X'] is not None else None
        y = int(str(a['Coordenada Y']).strip()) if a['Coordenada Y'] is not None else None
        point=transform_coordinates(x, y,srid=srid) if a['Coordenada X'] is not None else None
        geometry=GeometryCollection()
        geometry.append(point) if a['Coordenada X'] is not None else None
        properties.append({"label":a['N. da Invasão'] if a.get('N. da Invasão') else a.get('Relatório Principal')})
    
        if a['Relatório Principal'] is not None:
            """Registro"""
            record=OccurrenceRecord(company=company,
    #                                 firm=engie_firm,
                                    firm=firm,
                                    status=ServiceOrderActionStatus.objects.get(name=a['Status'].replace('elaboração','Elaboração'), companies=company) if a.get('Status') else status,
                                    approval_step=ApprovalStep.objects.get(approval_flow__company=company, name=a['Status'].replace('elaboração','Elaboração')),
    #                                 datetime=a['Data do registro'].strftime('%Y-%m-%dT%H:%M:00-03:00'),
                                    datetime=a['Data do registro'] if a['Data do registro'] is not None and isinstance(a['Data do registro'],datetime) else datetime.now(),
                                    uf_code=get_uf(a['Município'].split('-')[1].strip()),
                                    city=City.objects.filter(name=a['Município'].split('-')[0].strip()).first(),
                                    occurrence_type=occ,
                                    created_by=user,
                                    geometry=geometry,
                                    properties=properties,
                                    search_tag_description = a.get('Relatório') if a.get('Relatório') else a.get('Relatório Principal'),
                                    origin="5",
                                    involved_parts=get_involved_parts(a),
                                    form_data={
                                        'irregularities': a['irreg'] if 'irreg' in a and a['irreg'] else None,
                                        'agreement_type': True if a['Houve acordo verbal?'] is not None and a['Houve acordo verbal?'].lower() == 'sim' else False,
                                        'deadline': a['Prazo de cumprimento'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Prazo de cumprimento'] is not None else None,
                                        'agreement': a['Acordo realizado com'] if a['Acordo realizado com'] is not None else None,
                                        'affected_area': a['Área afetada (m2)_1'] if a.get('Área afetada (m2)_1') and a.get('Área afetada (m2)_1') is not None else None,
                                        'observation': a['obs'] if 'obs' in a and a['obs'] else None
                                    }
                                   )
            if not record.form_data['observation']:
                del record.form_data['observation']

            if not record.form_data['irregularities']:
                del record.form_data['irregularities']
                
            property_intersections(record)

            record.save()        
            record.search_tags.add(*[tag1,tag3,tag4])
            """Serviço"""        
            service_order=ServiceOrder(company=company, description=(a['Descrição do serviço'] or "Teste"),
                                       kind="LAND",process_type="1", created_by=user)        
            service_order.save()

            record.service_orders.add(service_order)
            service_order.responsibles.add(engie_user)
    #         service_order.responsibles.add(user)

            """Entregas"""
            for index, item in enumerate(get_action_list(actions, a['Entrega Atual'])):

                service_order_action=ServiceOrderAction(name=item['name'],
                                                        estimated_end_date=date.today()+timedelta(weeks=item['timedelta']['weeks']),
                                                        created_by=user,
                                                        service_order=service_order)            
                service_order_action.save()

                """Tarefa"""
                if index == 0 and a['Tarefa atual']:
                    procedure=Procedure(action=service_order_action,
                                        responsible=engie_user,
#                                         responsible=user,
                                        firm=firm,
    #                                     firm=engie_firm
                                        to_do=a['Tarefa atual'],
                                        created_by=user,
                                        deadline=datetime.strptime(deadline, "%d/%m/%Y %H:%M")
                                       )


                    procedure.save()
            file_para_re = path+str(a.get('Relatório Principal'))
                    
        elif a['Relatório Principal'] is None:
            recordRe=OccurrenceRecord(company=company,
                                    firm=firm,
                                    status=ServiceOrderActionStatus.objects.get(name=a['Status'].replace('elaboração','Elaboração'), companies=company),
                                    approval_step=ApprovalStep.objects.get(approval_flow__company=company, name=a['Status'].replace('elaboração','Elaboração')),
    #                                 datetime=a['Data do registro'].strftime('%Y-%m-%dT%H:%M:00-03:00'),
                                    datetime=a['Data do registro'] if a['Data do registro'] is not None and isinstance(a['Data do registro'],datetime) else values[282]['Data do registro'],
                                    uf_code=get_uf(a['Município'].split('/')[1]),
                                    city=City.objects.filter(name=a['Município'].split('/')[0]).first(),
                                    occurrence_type=occ_RE,
                                    search_tag_description = a.get('Relatório',''),
                                    created_by=user,
                                    geometry=geometry,
                                    properties=properties,
                                    origin="5",
                                    involved_parts=get_involved_parts(a),
                                   )
            recordRe.save()
            recordRe.search_tags.add(*[tag10,tag30,tag40])
            try:
                recordRe.service_orders.add(service_order)
                file=File(content_type=ContentType.objects.get(app_label='occurrence_records',model='occurrencerecord'),
                    object_id=str(recordRe.uuid),
                    content_object = recordRe,
                    created_by=user,
                    upload=file_para_re+'.pdf')
                file.save()
            except:
                pass
        else:
            print(a['Relatório Principal'],'Linha: ',i+2)
            erros.append(a)
            
    except Exception as e:
        print(f'ERRO - {e}{record.form_datas}')

  0%|          | 0/294 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2015-04-16 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2015-04-16 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ServiceOrder.opened_at received a naive datetime (2024-03-22 08:16:56.849122) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive

Procedure matching query does not exist.


WARNING	Property: Unknown Property name. [1:1: word-wrap]


'NoneType' object has no attribute 'is_final'


...

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2020-11-16 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2020-11-16 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ServiceOrder.opened_at received a naive datetime (2024-03-22 08:23:23.731036) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive

Procedure matching query does not exist.


WARNING	Property: Unknown Property name. [1:1: word-wrap]


'NoneType' object has no attribute 'is_final'


AttributeError: 'OccurrenceRecord' object has no attribute 'form_datas'

In [36]:
# procedure.delete(),service_order_action.delete(),service_order.delete(),record.delete()

((1, {'service_orders.Procedure': 1}),
 (1, {'service_orders.ServiceOrderAction': 1}),
 (14,
  {'occurrence_records.OccurrenceRecord_service_orders': 1,
   'service_orders.ServiceOrder_responsibles': 1,
   'service_orders.ServiceOrderAction': 11,
   'service_orders.ServiceOrder': 1}),
 (4,
  {'occurrence_records.OccurrenceRecord_search_tags': 3,
   'occurrence_records.OccurrenceRecord': 1}))

In [50]:
len(get_action_list(actions, a['Entrega Atual']))*376

4512

In [51]:
data_ref='2024-03-22'
pro=Procedure.objects.filter(firm__company=company,created_by=user,created_at__gte=data_ref)
so=ServiceOrder.objects.filter(company=company,created_by=user,updated_at__gte=data_ref)
soa=ServiceOrderAction.objects.filter(service_order__company=company,created_by=user,service_order__in=so)
rec = OccurrenceRecord.objects.filter(company=company,created_by=user,created_at__gte=data_ref)
pro.count(),soa.count(),so.count(),rec.count()

(369, 4428, 369, 369)

In [27]:
# pro.delete()
# soa.delete()
# so.delete()
# rec.delete()

(44,
 {'occurrence_records.OccurrenceRecord_search_tags': 33,
  'occurrence_records.OccurrenceRecord': 11})

In [122]:
# srid = 31982 if '22' in str(22) else 31983
point=transform_coordinates(185226, 7818142,srid=31983)
geometry=GeometryCollection()
geometry.append(point)

In [123]:
point.x,point.y

(-48.002576235169265, -19.707148876489743)

In [124]:
ocr=OccurrenceRecord.objects.get(number='UHMI-RG-2023.1064',company__name='GES - UHE-MIRANDA')

In [125]:
ocr.point = point
ocr.geometry = geometry

In [126]:
ocr.save()

In [30]:
ocr.geometry.__dict__

{'_constructor_args': ((), {}),
 '_ptr': <django.contrib.gis.geos.libgeos.LP_GEOSGeom_t at 0x7f691ccb9710>,
 '_cs': None}

In [34]:
properties.append({"label":a['N. da Invasão']})

7783921

In [26]:
rep_edit=[]
erros=[]
for a in tqdm(values):
    
    """Geometria"""
    properties=[]
    if a['Coordenada X'] is not None and '23' in str(a['Fuso']).strip() :
        point=transform_coordinates(a['Coordenada X'], a['Coordenada Y'],srid=31983)
    else:
        point=transform_coordinates(a['Coordenada X'], a['Coordenada Y']) if a['Coordenada X'] is not None else None
#     point=transform_coordinates(a['Coordenada X'], a['Coordenada Y']) if a['Coordenada X'] is not None else None
    geometry=GeometryCollection()
    geometry.append(point) if a['Coordenada X'] is not None else None
#     properties.append({})
    try:
        record=OccurrenceRecord.objects.get(company=company,
                                    firm=firm,
                                    status=ServiceOrderActionStatus.objects.get(name=a['Status'].replace('elaboração','Elaboração'), companies=company),
                                    approval_step=ApprovalStep.objects.get(approval_flow__company=company, name=a['Status'].replace('elaboração','Elaboração')),
                                    datetime=a['Data do registro'] if a['Data do registro'] is not None else None,
                                    uf_code=get_uf(a['Município'].split('/')[1]),
                                    city=City.objects.get(name=a['Município'].split('/')[0]),
                                    occurrence_type=occ,
                                    created_by=user,
                                    geometry=geometry,
#                                     properties=properties,
                                    origin="5")
        record.properties=[{"label":a['N. da Invasão']}]
        
        rep_edit.append(a)
    except Exception as e:
        erros.append(a)
        print(e)

  0%|          | 0/1025 [00:00<?, ?it/s]

get() returned more than one OccurrenceRecord -- it returned 10!
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matchi

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not

OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.


In [25]:
len(rep_edit),len(erros)

(332, 693)

In [85]:
for a in tqdm(values):

    try:
        record=OccurrenceRecord.objects.get(company=company,
                                    firm=firm,
                                    status=ServiceOrderActionStatus.objects.get(name=a['Status'].replace('elaboração','Elaboração'), companies=company),
                                    approval_step=ApprovalStep.objects.get(approval_flow__company=company, name=a['Status'].replace('elaboração','Elaboração')),
                                    datetime=a['Data do registro'] if a['Data do registro'] is not None else None,
                                    uf_code=get_uf(a['Município'].split('/')[1]),
                                    city=City.objects.get(name=a['Município'].split('/')[0]),
                                    occurrence_type=occ,
                                    created_by=user,
                                    created_at__gte='2023-10-28',
                                    origin="5")

        record.properties=[{"label":a['N. da Invasão']}]
        record.save()
    except Exception as e:
        erros.append(a)
        continue
    

  0%|          | 0/395 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [84]:
len(erros),erros[0]

(342,
 {'N. da Invasão': 'UHJA-086E',
  'Região': 'Orla',
  'Fuso': 23,
  'Coordenada X': 246661,
  'Coordenada Y': 7777363,
  'Município': 'Rifaina/SP',
  'Status': 'Registro em elaboração',
  'Equipe': 'LR Ambiental',
  'Data do registro': datetime.datetime(2005, 6, 22, 0, 0),
  'Classe': 'Uso e Ocupação Irregular',
  'Identificação do local_1': 'Área de inundação',
  'Identificação do local_2': None,
  'Identificação do local_3': None,
  'Identificação do local_4': None,
  'Identificação do local_5': None,
  'Identificação do local_6': None,
  'Identificação do local_7': None,
  'Identificação do local_8': None,
  'Constatação_1': 'Ocupação/utilização não autorizada',
  'Constatação_2': None,
  'Constatação_3': None,
  'Constatação_4': None,
  'Constatação_5': None,
  'Constatação_6': None,
  'Constatação_7': None,
  'Constatação_8': None,
  'Constatação (detalhe)_1': 'Muro',
  'Constatação (detalhe)_2': None,
  'Constatação (detalhe)_3': None,
  'Constatação (detalhe)_4': None,
  '

In [16]:
occs=OccurrenceRecord.objects.filter(created_at__gte='2023-11-26',company=company,occurrence_type=occ)

In [15]:
for a in occs:
    a.search_tags.add([tag1,tag3,tag4])

TypeError: 'OccurrenceRecord' object is not iterable

In [14]:
occ

<OccurrenceType: Ocorrências patrimoniais - ['GES - UHE-SALTO SANTIAGO', 'GES - UHE-ITA', 'GES - GEH', 'GES - UHE-ESTREITO', 'GES - UHE-JAGUARA', 'GES - UHE-MIRANDA', 'GES - UHE-SALTO OSORIO', 'GES - UHE-CANA BRAVA', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-PASSO FUNDO', 'GES - UHE-MACHADINHO', 'GES - UHE-PONTE DE PEDRA', 'GES - PCH-JOSÉ GELÁZIO DA ROCHA', 'GES - PCH-RONDONÓPOLIS']>

In [1]:
ocr=OccurrenceRecord.objects.get(uuid='d8eeb3a8-55f2-48b9-9034-00359e49af55')

In [6]:
ocr.search_tags.all()

<QuerySet [<SearchTag: GES - UHE-SALTO SANTIAGO - 2 - select_option - Resumo executivo>, <SearchTag: GES - UHE-SALTO SANTIAGO - 4 - select_option - Ictiofauna>, <SearchTag: GES - UHE-SALTO SANTIAGO - 1 - select_option - Relatório>, <SearchTag: GES - UHE-SALTO SANTIAGO - 3 - select_option - Ambiental>]>

In [79]:
# ocr.save() RO250200440-11

In [1]:
File._meta.get_fields()

(<GenericRel: occurrence_records.occurrencerecord>,
 <GenericRel: monitorings.monitoringrecord>,
 <GenericRel: monitorings.operationalcontrol>,
 <GenericRel: monitorings.operationalcontrolrecord>,
 <GenericRel: constructions.construction>,
 <GenericRel: constructions.constructionprogress>,
 <django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.TextField: description>,
 <django.db.models.fields.TextField: md5>,
 <django.db.models.fields.files.FileField: upload>,
 <django.db.models.fields.DateTimeField: uploaded_at>,
 <django.db.models.fields.DateTimeField: datetime>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.CharField: kind>,
 <django.db.models.fields.TextField: url>,
 <django.db.models.fields.related.ForeignKey: content_type>,
 <django.db.models.fields.UUIDField: object_id>,
 <django.contrib.contenttypes.fields.GenericForeignKey at 0x7f1412e62350>)

In [11]:
file=File(content_type=ContentType.objects.get(app_label='occurrence_records',model='occurrencerecord'),
     object_id=str(ocr.uuid),
    field.content_object = ocr,
          created_by=
     upload='/uhss_importacao/RO250200440-11.pdf')

In [16]:
field.save()

In [13]:
field.content_object = ocr
field.save()

In [15]:
field.upload='RO250200440-11.pdf'

In [17]:
field.delete()

(1, {'files.File': 1})

In [6]:
user=User.objects.filter(username='engie')

In [9]:
Firm.objects.all()

<QuerySet [<Firm: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] 7047df2f-8501-4d0d-a4db-6f724def28c4: ENGIE Brasil Energia>, <Firm: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] 6e4a34e4-fbaa-4953-8bb2-cd91b5a68f7f: Cadastro de rotas de fuga>, <Firm: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] 1c4a0a75-21f5-426f-a32c-83c37e9e7a54: Cadastro de pontos de encontro>, <Firm: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] 87476923-b71f-4190-9606-ba658bcb2baa: Equipe Sonora>, <Firm: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] f76b8aa4-71fa-438a-9f4d-4a1c53461d5b: Equipe Itiquira>, <Firm: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] 622dbd8b-74a6-4ab1-b174-1b13b3f57ec2: Administração>, <Firm: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] 2918a1f4-3e9b-4860-82af-b30a0c091ed3: Suporte>, <Firm: [110bce05-5146-421e-a5a7-ffb32ee92a91: GSO - UHE-PASSO FUNDO] 4a25e8

In [2]:
import json
for instance in OccurrenceRecord.objects.filter(created_at__gte='2023-11-27',created_at__lte='2023-11-28',
                                         company__name='GES - UHE-SALTO SANTIAGO')[1:]:
    geom = instance.geometry

    if geom and "properties_shape" in instance.company.metadata:
        try:
            shape = ShapeFile.objects.get(
                uuid=instance.company.metadata["properties_shape"]
            )
        except Exception:
            pass
        else:
            intersects = []
            for index, geometry in enumerate(shape.geometry):
                try:
                    if geometry.intersects(geom):
                        intersects.append((index, geometry))
                except Exception:
                    pass

            intersects = [
                {
                    "geometry": json.loads(geometry[1].json),
                    "attributes": {
                        "uuid": "{}-{}".format(
                            str(shape.uuid),
                            shape.properties[geometry[0]]["OBJECTID"],
                        ),
                        **shape.properties[geometry[0]],
                    },
                }
                for geometry in intersects
            ]

            field = "property_intersections"
            instance.form_data[field] = intersects
            instance.save()
#             print(instance.number)

In [1]:
def property_intersections(instance):
    geom = instance.geometry

    if geom and "properties_shape" in instance.company.metadata:
        try:
            shape = ShapeFile.objects.get(
                uuid=instance.company.metadata["properties_shape"]
            )
        except Exception:
            pass
        else:
            intersects = []
            for index, geometry in enumerate(shape.geometry):
                try:
                    if geometry.intersects(geom):
                        intersects.append((index, geometry))
                except Exception:
                    pass

            intersects = [
                {
                    "geometry": json.loads(geometry[1].json),
                    "attributes": {
                        "uuid": "{}-{}".format(
                            str(shape.uuid),
                            shape.properties[geometry[0]]["OBJECTID"],
                        ),
                        **shape.properties[geometry[0]],
                    },
                }
                for geometry in intersects
            ]

            field = "property_intersections"
            instance.form_data[field] = intersects

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.DateTimeField: datetime>,
 <django.db.models.fields.CharField: number>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.DateTimeField: updated_at>,
 <django.db.models.fields.IntegerField: reviews>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.BooleanField: editable>,
 <django.db.models.fields.BooleanField: is_approved>,
 <django.db.models.fields.IntegerField: uf_code>,
 <django.db.models.fields.related.ForeignKey: city>,
 <django.db.models.fields.related.ForeignKey: location>,
 <django.db.models.fields.CharField: place_on_dam>,
 <django.db.models.fields.related.ForeignKey: river>,
 <django.contrib.gis.db.models.fields.PointField: point>,
 <django.contrib.gis.db.models.fields.GeometryCollectionField: geometry>,
 <django.contrib.postgres.fields.jsonb.JSONField: properties>,
 <django.db.models.fields.FloatField: distance_from_dam>,
 <django.db.models.f

In [12]:
occ=OccurrenceType.objects.get(uuid='faa62cbd-c2ec-49fc-bc4a-9b1ab9edcca4')# Ocorrências Patrimoniais
occ_RE=OccurrenceType.objects.get(uuid='df7b367d-893b-40cc-af39-2019a2e764b9')# Resumo Executivo
company = Company.objects.get(name='GES - UHE-JAGUARA')
occurrence_type=OccurrenceType.objects.get(uuid='f200613b-8862-4a24-8c16-d6babd215fd7')# matriz vegetal
species_type=OccurrenceType.objects.get(uuid='c16b1d6b-856c-4ff7-ae2f-f68d5223a077')
operational_control=OperationalControl.objects.get(uuid='d0b541b7-713b-4454-8fac-59ddbb937d90')# monitoramento hidrogeológico
occ.name,occ_RE.name,company.name,occurrence_type.name

('Ocorrências patrimoniais',
 'Resumo Executivo',
 'GES - UHE-JAGUARA',
 'Cadastro de Matriz Vegetal')

In [16]:
ocr=OccurrenceRecord.objects.filter(created_at__gte='2023-10-28',created_at__lte='2023-10-29',
                                    occurrence_type=occ_RE,
#                                     operational_control=operational_control,
                                         company=company)
ocr.count()

0

In [48]:
tag10=SearchTag.objects.get(company = company, name = "Relatório")
tag30=SearchTag.objects.get(company = company, name = "Resumo executivo")
tag40=SearchTag.objects.get(company = company, name = "Patrimonial")
tag10.name,tag30.name,tag40.name

('Relatório', 'Resumo executivo', 'Patrimonial')

In [49]:
for a in tqdm(ocr):
    a.search_tags.add(*[tag10,tag30,tag40])

  0%|          | 0/1364 [00:00<?, ?it/s]

In [14]:
tag1=SearchTag.objects.get(company = company, name = "Ocorrência")
tag3=SearchTag.objects.get(company = company, name = "Patrimonial")
tag4=SearchTag.objects.get(company = company, name = "Uso e ocupação irregular")
tag1.name,tag3.name,tag4.name

('Ocorrência', 'Patrimonial', 'Uso e ocupação irregular')

In [15]:
for a in tqdm(ocr):
    a.search_tags.add(*[tag1,tag3,tag4])

  0%|          | 0/395 [00:00<?, ?it/s]

In [39]:
ocr[0].number

'UHMI-RG-2023.0937'

In [69]:
tag=SearchTag.objects.get(company = company, name = "Controle Operacional")
tag.name

'Controle Operacional'

In [70]:
for a in tqdm(ocr):
    a.search_tags.add(*[tag])

  0%|          | 0/2719 [00:00<?, ?it/s]

In [71]:
ocr[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7ff1f3c048d0>,
 'uuid': UUID('6d5edef7-6a21-41a5-b138-a8fd176a3ce8'),
 'datetime': datetime.datetime(2024, 1, 27, 1, 9, 30, 257678, tzinfo=<UTC>),
 'number': '',
 'created_at': datetime.datetime(2024, 1, 27, 1, 14, 33, 466759, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2024, 1, 27, 1, 14, 33, 466785, tzinfo=<UTC>),
 'reviews': 0,
 'company_id': UUID('42ddc389-3258-4daf-868a-097235427e0f'),
 'editable': True,
 'is_approved': False,
 'uf_code': 21,
 'city_id': UUID('5238f4ca-08fa-41c9-b1d2-ffccc4dc3d43'),
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': <Point object at 0x7ff1f2704be0>,
 'geometry': <GeometryCollection object at 0x7ff1f2704d00>,
 'properties': [],
 'distance_from_dam': 0.0,
 'other_reference': '',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'search_tag_description': '',
 'occurrence_